In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd
import datetime


In [4]:
###Load the trained model
model =load_model('model.h5')

##Load the encoder and scaler file
with open('onehot_encoder_geo.pkl','rb') as f:
    label_encoder_geo = pickle.load(f)


with open('label_encoder_gender.pkl','rb') as f:
    label_encoder_gender = pickle.load(f)

with open('scaler.pkl','rb') as f:
    scaler = pickle.load(f)    

In [6]:
#emaple input data
input_data ={
    'CreditScore': 600,
    'Geography': 'France', 
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000

}

In [8]:
##One hot encode the geo
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/puneetpurohit/ANNProject/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:

input_df= pd.DataFrame([input_data])


In [12]:
##Encode categorical variables
input_df['Gender'] =label_encoder_gender.transform(input_df['Gender'])


In [13]:
##COntatenation of one hot encoded columns with input df
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
###Scaling the data

# Add missing columns with dummy values if not present
for col in ['RowNumber', 'CustomerId']:
    if col not in input_df.columns:
        input_df[col] = 0  # or any integer

# Reorder columns to match scaler's fit order
input_df = input_df[scaler.feature_names_in_]
input_data_scaled =scaler.transform(input_df)
input_data_scaled

array([[-1.73595179e+00, -2.18402312e+02, -5.35985161e-01,
         9.13247552e-01,  1.04793593e-01, -6.95393486e-01,
        -2.57811188e-01,  8.08436154e-01,  6.49202671e-01,
         9.74816989e-01, -8.76832208e-01,  1.00150113e+00,
        -5.79467227e-01, -5.76388018e-01]])

In [17]:
##Predict churn
prediction = model.predict(input_data_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[2.2709339e-21]], dtype=float32)

In [19]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(2.2709339e-21)

In [20]:
if prediction_prob > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_prob:.2f}")
else:
    print(f"The customer is not likely to churn with a probability of {prediction_prob:.2f}")

The customer is not likely to churn with a probability of 0.00
